In [ ]:
import collections
import sys
import subprocess
import pandas as pd
import os
import numpy as np
import uuid
import tempfile
import bnpy
import scikit_posthocs as sp
import itertools

from scipy.stats import kruskal, f_oneway
from scipy.stats import ttest_ind

sys.path.append('/opt/hydra/')

src = os.environ["HYDRA_SRC"]

import library.analysis as hy

In [ ]:
# This cmd was used to identify multimodal genes 
cmd = ["docker" ,
       "run",
       "-v", "$PWD:/data",
       "jpfeil/hydra@sha256:123bee0aa2b3e63084c773a13a16d247076462af910a104cd5776ba5e6d4b29d",
       "filter",
       "-e", "data/TARGET-osteosarcoma-2019-02-02.tsv",
       "--output-dir", "output",
       "--CPU", "13"]

In [ ]:
mm_genes = '../data/output/MultiModalGenes/'
exp_path = '../data/TARGET-osteosarcoma-2019-02-02.tsv'

In [ ]:
exp = pd.read_csv(exp_path, sep='\t', index_col=0)

In [ ]:
res = hy.EnrichmentAnalysis(exp_path=exp_path,
                            mm_path=mm_genes,
                            min_prob_filter=0.1,
                            min_effect_filter=1.0,
                            gmt_path='GO')

In [ ]:
res.get_enriched_terms()

In [ ]:
genes = res.get_enriched_term_genes(regex='muscle')

len(genes)

In [ ]:
clus = hy.MultivariateMixtureModel(data=exp.reindex(genes),
                                   center=True,
                                   gamma=5.0,
                                   variance=2.0,
                                   K=2, 
                                   verbose=True)

In [ ]:
assignments = clus.get_assignments(exp.reindex(genes))

assign = pd.DataFrame(index=exp.columns,
                      columns=[1])

for sample, assignment in zip(exp.columns, assignments):
    assign.loc[sample, 1] = assignment + 1

In [ ]:
hclust = hy.HClust(data=exp.reindex(genes))

In [ ]:
hclust.plot_row_linkage(17)
row_groups = hclust.get_row_groups(17)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

#col_order = assign.sort_values(1).index.values

g = sns.clustermap(exp.reindex(genes).dropna(),
                   z_score=0,
                   method='ward',
                   center=0,
                   cmap=sns.diverging_palette(240, 10, n=7),
                   figsize=(10, 10))

ax = g.ax_heatmap

ax.set_xticklabels([])
ax.set_xticks([])
ax.set_yticklabels([])
ax.set_yticks([])

pth = '../img/expression-heatmap.svg'
plt.savefig(pth, format='svg', bbox_inches='tight')

In [ ]:
help(hy.n1)

In [ ]:
fgseas = {}
for cluster, rows in assign.groupby(1):
    ins = rows.index.values
    outs = [x for x in exp.columns if x not in ins]
    
    r = ttest_ind(exp[ins].values,
                  exp[outs].values,
                  axis=1).statistic
                    
    tstats = pd.DataFrame(index=exp.index, 
                          data=r).dropna()
                          
    tstats = tstats.sort_values(0, ascending=False)
    
    fgseas[cluster] = hy.n1(tstats)

In [ ]:
t = fgseas[2]

t[(t['padj'] < 0.05) & (t['ES'] > 0)].sort_values('NES', ascending=False)

In [ ]:
pth = '../data/xCell_TARGET-osteosarcoma-2019-02-02_xCell_1553020819.txt'
xcell = pd.read_csv(pth, sep='\t', index_col=0)
xcell.columns = [x.replace('.', '-') for x in xcell.columns]

In [ ]:
igroups = dict((group, []) for group in assign[1].unique())
data = pd.DataFrame(columns=['sample', 
                             'cluster', 
                             'source', 
                             'feature', 
                             'value'])

for sample in assign.index.values:
  
    assignment = assign.loc[sample, 1]
    
    if pd.isnull(assignment):
        continue
    
    igroups[assignment].append(sample)
        
    for cell, value in xcell[sample].iteritems():
        data.loc[len(data), :] = [sample, assignment, 'xCell', cell, value]
        
skip = ['P-value', 'Pearson Correlation', 'RMSE']

data = data[~data['feature'].isin(skip)]
        
data['value'] = pd.to_numeric(data['value'])

In [ ]:
data

In [ ]:
from scipy.stats import kruskal, f_oneway
import collections
import scikit_posthocs as sp
import itertools

sigs = collections.defaultdict(list)

p_values = pd.DataFrame(columns=['source', 
                                 'feature', 
                                 'A', 
                                 'B', 
                                 'holm p-value'])

for source, rows in data.groupby('source'):
    print(source)
    print()
    nfeatures = len(rows['feature'].unique())
    alpha = 1 - (1 - 0.05) ** (1.0 / nfeatures)
    print(alpha)
    for feature in rows['feature'].unique():
        print(feature)
        groups3 = []
        for i, rows in data[data['feature'] == feature].groupby('cluster'):
            groups3.append(list(rows['value'].values))
    
        try:
            stat, pvalue = kruskal(*groups3)
            print(feature, pvalue, pvalue < alpha)
        
        except:
            continue
    
        if pvalue < alpha:
            sigs[source].append(feature)
            
            res = sp.posthoc_mannwhitney(groups3, 
                                         p_adjust='holm')
            
            for i, j in itertools.combinations(range(len(groups3)),
                                               2):
                
                if i == j:
                    continue 
                    
                p_values.loc[len(p_values), :] = [source,
                                                  feature,
                                                  i, 
                                                  j,
                                                  res[i, j]]

In [ ]:
mask = (data['source'] == 'xCell') #& (data['feature'].isin(sigs['xCell']))

t = data[mask]

g = sns.catplot(x='cluster', 
                y='value', 
                col='feature',
                kind='box',
                col_wrap=3,
                sharex=False,
                sharey=False,
                data=t)

for i in range(len(g.axes)):
    g.axes[i].set_xlabel('Hydra Cluster')
    g.axes[i].set_ylabel('xCell Enrichment Score')
    
plt.subplots_adjust(hspace=0.3, wspace=0.4)

In [ ]:
sns.set(style='white', font_scale=1.5)

mask = (data['source'] == 'xCell') & (data['feature'].isin(['Skeletal muscle']))

t = data[mask]

t.loc[t['cluster'].isin([1]), 'merge'] = 'Background'
t.loc[t['cluster'].isin([2]), 'merge'] = 'Muscle Cluster'

fig, ax = plt.subplots(1, 
                       figsize=(5, 5))

g = sns.boxplot(x='merge',
                y='value',
                data=t,
                color='white',
                ax=ax)

sns.swarmplot(x='merge',
              y='value',
              data=t,
              color='k',
              size=5,
              ax=ax)

plt.subplots_adjust(hspace=0.3, wspace=0.4)

g.set_xlabel('')
g.set_ylabel('xCell Score')

pth = '../img/osteo-muscle-xcell.svg'
plt.savefig(pth, format='svg', bbox_inches='tight')

pth = '../img/osteo-muscle-xcell.png'
plt.savefig(pth, format='png', bbox_inches='tight')